In [1]:
import os 
import numpy as np
import cv2
import pandas as pd
from PIL import ImageFont, ImageDraw, Image
import jsonlines
import PyPDF2
import pdf2image
from PyPDF2 import PdfFileReader
from matplotlib.patches import  Rectangle

In [2]:
data_fintabnet_path =  "D:/fintabnet/fintabnet/"
subdir = "train"
structure_filename  = "FinTabNet_1.0.0_cell_" + subdir + ".jsonl"
detection_filename = "FinTabNet_1.0.0_table_" + subdir +  ".jsonl"
structure_filepath = os.path.join(data_fintabnet_path, structure_filename)
detection_filepath = os.path.join(data_fintabnet_path, detection_filename)
print(f'detection_filepath :{structure_filepath}')
with open(structure_filepath, "r") as f:
    structure_lines = f.readlines()

detection_filepath :D:/fintabnet/fintabnet/FinTabNet_1.0.0_cell_train.jsonl


In [18]:
i = 0
data_frame = {}
with jsonlines.open(detection_filepath,'r') as f:
# with jsonlines.open('FinTabNet_1.0.0_table_example.jsonl','r') as f:
    for data in f:
        if i > 30000 :
            break
        file_name = data['filename']
        print(f'file_name :{file_name}')
        if file_name in data_frame:
            box     = data['bbox']
            data_frame[file_name].append(box) 
        else:
            box = data['bbox']  
            update = {file_name : [box]}
            data_frame.update(**update)
        i = i +1 

file_name :MDT/2018/page_32.pdf
file_name :MDT/2018/page_34.pdf
file_name :MDT/2018/page_36.pdf
file_name :MDT/2018/page_48.pdf
file_name :MDT/2018/page_49.pdf
file_name :MDT/2018/page_51.pdf
file_name :MDT/2018/page_53.pdf
file_name :MDT/2018/page_54.pdf
file_name :MDT/2018/page_60.pdf
file_name :MDT/2018/page_61.pdf
file_name :MDT/2018/page_64.pdf
file_name :MDT/2018/page_74.pdf
file_name :MDT/2018/page_93.pdf
file_name :MDT/2018/page_95.pdf
file_name :MDT/2018/page_99.pdf
file_name :MDT/2018/page_100.pdf
file_name :MDT/2018/page_108.pdf
file_name :MDT/2018/page_117.pdf
file_name :MDT/2015/page_46.pdf
file_name :MDT/2015/page_63.pdf
file_name :MDT/2015/page_66.pdf
file_name :MDT/2015/page_67.pdf
file_name :MDT/2015/page_67.pdf
file_name :MDT/2015/page_69.pdf
file_name :MDT/2015/page_70.pdf
file_name :MDT/2015/page_72.pdf
file_name :MDT/2015/page_77.pdf
file_name :MDT/2015/page_78.pdf
file_name :MDT/2015/page_81.pdf
file_name :MDT/2015/page_92.pdf
file_name :MDT/2015/page_92.pdf
file_

In [20]:
data_frame

{'MDT/2018/page_32.pdf': [[41.9527, 355.06699999999984, 379.7047, 539.2347]],
 'MDT/2018/page_34.pdf': [[41.9527, 378.4357, 549.3895, 429.7434]],
 'MDT/2018/page_36.pdf': [[41.94885999999997,
   149.0453,
   549.5685799999998,
   635.7544]],
 'MDT/2018/page_48.pdf': [[43.6851,
   521.3399999999999,
   551.6935799999998,
   584.2008999999999]],
 'MDT/2018/page_49.pdf': [[44.7874, 531.0406000000003, 550.92996, 688.0548]],
 'MDT/2018/page_51.pdf': [[44.7875,
   354.9287999999999,
   556.0413000000001,
   476.8737]],
 'MDT/2018/page_53.pdf': [[44.785999999999945,
   557.1733,
   553.0061600000001,
   656.8126]],
 'MDT/2018/page_54.pdf': [[41.95187999999996,
   485.6689999999999,
   549.75652,
   657.5509]],
 'MDT/2018/page_60.pdf': [[41.9528,
   250.79050000000018,
   550.9883200000006,
   645.1167]],
 'MDT/2018/page_61.pdf': [[44.7872999999999,
   488.10250000000013,
   553.69738,
   645.1167]],
 'MDT/2018/page_64.pdf': [[41.944860000000006,
   48.79309999999991,
   550.8446,
   645.1167]

In [22]:
final_data = pd.DataFrame(data_frame.items())
final_data.to_csv("YOLO_data.csv", index=False)
row, col  = final_data.shape
row

23305

In [30]:
data_path =         "D:/fintabnet/fintabnet/pdf/"
data_saved_path =   "C:/Users/PC/Desktop/git/YOLO_TD/data_yolo_fintab/"
image_train_path =  "C:/Users/PC/Desktop/git/YOLO_TD/data_yolo_fintab/images/train/"
label_train_path =  "C:/Users/PC/Desktop/git/YOLO_TD/data_yolo_fintab/labels/train/"

In [24]:
def normalize_img(img):
    #norm_img = (img - img.min()) / (img.max() - img.min())
    norm_img = np.array(img,dtype= np.float32)/255.0
    return norm_img

def get_table_bbox(pdf_height, pdf_width, table_bbox, HEIGHT = 1024, WIDTH = 1024):
    tl_x = int(table_bbox[0])
    tl_y = int(pdf_height - table_bbox[3])
    br_x = int(table_bbox[2])
    br_y = int(pdf_height - table_bbox[1])
    new_tl_x = int(tl_x * WIDTH / pdf_width)
    new_tl_y = int(tl_y * HEIGHT / pdf_height)
    new_br_x = int(br_x * WIDTH / pdf_width)
    new_br_y = int(br_y * HEIGHT / pdf_height)
    return (new_tl_x, new_tl_y, new_br_x, new_br_y)
def gen_image(input_pdf_path):
    pdf = PdfFileReader(input_pdf_path)
    pdf_shape = pdf.getPage(0).mediaBox
    pdf_height = pdf_shape[3]-pdf_shape[1]
    pdf_width = pdf_shape[2]-pdf_shape[0]
    converted_images = pdf2image.convert_from_path(input_pdf_path, size=(pdf_width, pdf_height))
    img = converted_images[0]
    img = np.array(img)
    return img
def convert_to_YOLO_format(x_tl, y_tl, x_br, y_br, IMG_WIDTH= 1024, IMG_HEIGHT = 1024):
    x_center = (x_tl + x_br)/2
    y_center = (y_tl + y_br)/2
    width = abs(x_tl - x_br)
    height = abs(y_tl - y_br)

    x_center = x_center / IMG_WIDTH
    y_center = y_center / IMG_HEIGHT
    width = width / IMG_WIDTH
    height = height / IMG_HEIGHT
    return x_center, y_center, width, height

In [25]:
def convert_coordinates(size, box):
    dw = 1.0/size[0]
    dh = 1.0/size[1]
    x = (box[0]+box[1])/2.0
    y = (box[2]+box[3])/2.0
    w = box[1]-box[0]
    h = box[3]-box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

In [31]:
def generate_data(image_input_path, detection , train = True , val = False):
    file_name = os.path.splitext(image_input_path)[0]
    image_input =  gen_image(data_path + image_input_path)



    file_name = file_name.replace('/', '_')
    file_name_txt = file_name+'.txt'
    file_name_jpg = file_name+'.jpg'

    if train:
        image_path = data_saved_path + 'images/train/'
        label_path = data_saved_path + 'labels/train/'
    if val:
        image_path = data_saved_path + 'images/val/'
        label_path = data_saved_path + 'labels/val/'
    label_save_path = label_path+ file_name_txt
    shape = image_input.shape
    h = shape[0]
    w = shape[1]
    image_ = cv2.resize(image_input, (1024,1024))
    image_ = image_.astype('float')
    image = normalize_img(image_)
    image = image_.astype(np.uint8)
    image = Image.fromarray(image)
    image.save(image_path +  file_name_jpg)

    with_ratio   = 1 # OUT_SIZE /IN_SIZE
    height_ratio = 1 # OUT_SIZE /IN_SIZE
    with open(label_save_path, 'w') as f:
        for idx, corner in enumerate(detection):
            corner = get_table_bbox(h,w,corner)
            x_tl_ori, y_tl_ori ,x_br_ori, y_br_ori = corner
            fx_tl = (x_tl_ori*with_ratio)
            fy_tl = (y_tl_ori*height_ratio)
            fx_br = (x_br_ori*with_ratio)
            fy_br = (y_br_ori*height_ratio)
            x_tl = int(fx_tl)
            y_tl = int(fy_tl)
            x_br = int(fx_br)
            y_br = int(fy_br)
            b = (float(x_tl), float(x_br), float(y_tl), float(y_br))
            bb = convert_coordinates((1024,1024), (b))
            
            f.write(str(0) + " "+ " ".join([("%.6f" % a) for a in bb]) + '\n')  # write line to output file

        

In [32]:
idx = 1 
for i in range(len(final_data[0])):
    image_name = final_data[0][i]
    # file_name = str(os.path.splitext(image_name)[0])
    # file_name = file_name.replace('/', '_')
    detection = final_data[1][i]
    if i < 20000:
        gen_data = generate_data(image_name,detection, train = True, val = False)
    else :
        gen_data = generate_data(image_name,detection,train = Flase,  val = True)

NameError: name 'Flase' is not defined

In [34]:
for i in range(20000,len(final_data[0])):
    image_name = final_data[0][i]
    detection = final_data[1][i]
    gen_data = generate_data(image_name,detection,train = False,  val = True)

In [27]:
len(final_data[0])

23305